<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_4_First_2D_truss_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np

# ============================================================
# FEM Giàn 2D - Chỉ tính Lực dọc & Chuyển vị (Output đơn giản)
# ============================================================

def formStiffness2Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, xx, yy, EA):
    K = np.zeros((GDof, GDof), dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]

        # Ánh xạ bậc tự do (DOF) phần tử
        edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1

        xa = xx[n2-1] - xx[n1-1]
        ya = yy[n2-1] - yy[n1-1]
        L = np.sqrt(xa*xa + ya*ya)

        C = xa / L
        S = ya / L

        k1 = (EA / L) * np.array([
            [ C*C,  C*S, -C*C, -C*S],
            [ C*S,  S*S, -C*S, -S*S],
            [-C*C, -C*S,  C*C,  C*S],
            [-C*S, -S*S,  C*S,  S*S],
        ], dtype=float)

        K[np.ix_(edof, edof)] += k1

    return K


def solution(GDof, prescribedDof, stiffness, force, u_prescribed=None):
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribedDof), dtype=float)

    prescribed = np.array([d-1 for d in prescribedDof], dtype=int)
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    K_ff = stiffness[np.ix_(free, free)]
    K_fp = stiffness[np.ix_(free, prescribed)]
    F_f = force[free]

    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)

    return U


def calculate_results(numberElements, elementNodes, xx, yy, displacements, E, A):
    """
    Chỉ tính toán và hiển thị Lực dọc (Axial Force) đơn giản
    """
    axial_forces = np.zeros(numberElements, dtype=float)
    EA_val = E * A

    print("\n=== KẾT QUẢ PHẦN TỬ (LỰC DỌC) ===")
    print(f"{'Phần tử':<10} | {'Lực dọc (N)'}")
    print("-" * 30)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]
        edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1

        xa = xx[n2-1] - xx[n1-1]
        ya = yy[n2-1] - yy[n1-1]
        L = np.sqrt(xa*xa + ya*ya)

        C = xa / L
        S = ya / L

        u_e = displacements[edof]

        # Công thức tính trực tiếp lực dọc: N = (EA/L) * [-C -S C S] * u
        axial_forces[e] = (EA_val / L) * np.array([-C, -S, C, S], dtype=float) @ u_e

        # In ra kết quả đơn giản: ID và Lực
        print(f"{e+1:<10} | {axial_forces[e]:.4f}")

    return axial_forces


def outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof, force):
    R = stiffness @ displacements - force

    print("\n=== CHUYỂN VỊ (TẠI CÁC NÚT) ===")
    for i in range(GDof):
        val = displacements[i]
        if abs(val) < 1e-9: val = 0.0
        print(f"DOF {i+1}: U = {val:.6f}")

    print("\n=== PHẢN LỰC (TẠI LIÊN KẾT) ===")
    for d in prescribedDof:
        val = R[d-1]
        if abs(val) < 1e-9: val = 0.0
        print(f"DOF {d}: R = {val:.6f}")


# ============================================================
# Chương trình chính (Main Script)
# ============================================================

# 1. Vật liệu và Tiết diện
E = 30e6
A = 2.0
EA = E * A

# 2. Lưới phần tử
numberElements = 3
numberNodes = 4

elementNodes = np.array([
    [1, 2],
    [1, 3],
    [1, 4]
], dtype=int)

nodeCoordinates = np.array([
    [0.0,   0.0],
    [0.0, 120.0],
    [120.0, 120.0],
    [120.0, 0.0]
], dtype=float)

xx = nodeCoordinates[:, 0]
yy = nodeCoordinates[:, 1]

# 3. Khởi tạo bậc tự do
GDof = 2 * numberNodes
displacements = np.zeros(GDof, dtype=float)
force = np.zeros(GDof, dtype=float)

# 4. Tải trọng
force[1] = -10000.0

# 5. Tính ma trận độ cứng
stiffness = formStiffness2Dtruss(
    GDof, numberElements, elementNodes, nodeCoordinates, xx, yy, EA
)

# 6. Điều kiện biên
prescribedDof = list(range(3, 9))

# 7. Giải hệ phương trình
displacements = solution(GDof, prescribedDof, stiffness, force)

# 8. Xuất kết quả
outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof, force)
axial_forces = calculate_results(numberElements, elementNodes, xx, yy, displacements, E, A)


=== CHUYỂN VỊ (TẠI CÁC NÚT) ===
DOF 1: U = 0.004142
DOF 2: U = -0.015858
DOF 3: U = 0.000000
DOF 4: U = 0.000000
DOF 5: U = 0.000000
DOF 6: U = 0.000000
DOF 7: U = 0.000000
DOF 8: U = 0.000000

=== PHẢN LỰC (TẠI LIÊN KẾT) ===
DOF 3: R = 0.000000
DOF 4: R = 7928.932188
DOF 5: R = 2071.067812
DOF 6: R = 2071.067812
DOF 7: R = -2071.067812
DOF 8: R = 0.000000

=== KẾT QUẢ PHẦN TỬ (LỰC DỌC) ===
Phần tử    | Lực dọc (N)
------------------------------
1          | 7928.9322
2          | 2928.9322
3          | -2071.0678
